<a href="https://colab.research.google.com/github/trefftzc/cis677/blob/main/max_ind_set_thrust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
%%writefile max_ind_set_thrust.cu
/*
 *
 * Solve the Maximum Independent Set problem on GPUs using Thrust
 *
 * https://en.wikipedia.org/wiki/Independent_set_(graph_theory)
 * This code works for graphs of up to 32 nodes
 * The input is expected to be as follows:
 * The first line will contain n, the number of elements in the multiset
 * The remaining n lines will contain the adjacency matrix
 */
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/sequence.h>
#include <thrust/reduce.h>
#include <thrust/functional.h>
#include <thrust/execution_policy.h>
#include <thrust/transform.h>


#include <iostream>
#include <string>
#include <sstream>
#include <vector>

using namespace std;

const int MAX_GRAPH_SIZE = 32;
// This functions evaluates a particular subset of nodes.
// adjMat is the adjacency
// n is the size of the number of nodes in the graph
// value is an integer value. The binary code of value
//
struct evaluateSubSet
{

  const int n;
  const int *adjMat;

  evaluateSubSet(int _n,int *_adjMat) : n(_n),adjMat(_adjMat) {}
    __device__

int operator()(const int value ) {
  int  set[MAX_GRAPH_SIZE];
  unsigned int mask = 1;
  int bitCount = 0;
  for(int i = 0;i < n;i++) {
    if ((mask & value) != 0) {
      set[i] = 1;
      bitCount++;
    }
    else {
      set[i] = 0;
    }
    mask = mask * 2;
  }
  for(int i = 0;i < n;i++) {
    for(int j = i+1;j < n;j++) {
      if (set[i] == 1 && set[j] == 1 && adjMat[i*n+j] == 1) {
        return 0;
      }
    }
  }
  return bitCount;
}
};

// Print results
void printResults(unsigned int value,int n)
{
unsigned int mask = 1;

  cout << "An independent set with the maximum number of nodes is: " << endl;
  for(int i = 0;i < n;i++) {
    if ((mask & value) != 0) {
      cout << i << " ";
    }
    mask = mask * 2;
  }
  cout << endl;
}

// The main function
int main() {

  int n;

// The format of the input is
// an integer value n with the size of the multiset
// n integer values with the multiset

  thrust::host_vector < thrust::host_vector < int> > adjMat;
  string line;
  getline(cin,line);
  stringstream ss(line);
  ss >> n;
  // Read input line by line until EOF
  while (std::getline(cin, line)) {
        stringstream ss(line);
        thrust::host_vector < int > row;
        int num;
        while (ss >> num) {
          //cout << num << " ";
          row.push_back(num);
        }
        //cout << "Size of row: " << row.size() << endl;
        adjMat.push_back(row);
        //cout << endl;
  }
  cout << "The size of the graph is: " << n << endl;
  cout << "The number of rows in the adjacency matrix is: " << adjMat.size() << endl;
  cout << "The number of columns in the adjacency matrix is: " << adjMat[0].size() << endl;

// Print the adjacency matrix

  cout << "The adjacency matrix is : " << endl;
  for(int i = 0;i < n;i++) {
    for(int j = 0;j < n;j++) {
      cout << adjMat[i][j] << " ";
    }
    cout << endl;
  }
// Convert the adjacency matrix to a 1D array in the GPU
thrust::device_vector < int > device_adjMat_as_array(n*n);
for(int i = 0;i < n;i++) {
  for(int j = 0;j < n;j++) {
    device_adjMat_as_array[i*n+j] = adjMat[i][j];
  }
}
// Calculate the set of the power set
  unsigned int powerOf2 = 1;
  for(int i = 0;i < n;i++) {
    powerOf2 = powerOf2 * 2;
  }

  // is_solution will contain the results of the evaluation
  // It is allocated in the GPU memory
  thrust::device_vector < int > is_solution(powerOf2);

  // encodingOfSubset will contain the sequence of integer values
  // that encode all the possible subsets
  // It is allocated in the GPU memory
  thrust::device_vector < int > encodingOfSubset(powerOf2);

  //  The sequence function in Thrust initializes the array
  // encodingOfSubset with the sequence of integer values
  // from 0 to powerOf2-1
    thrust::sequence(encodingOfSubset.begin(),encodingOfSubset.end());



    // Instantiate the functor

    evaluateSubSet ef(n,thrust::raw_pointer_cast(device_adjMat_as_array.data()));
    // cout << "Instantiated functor " << endl;
    // Execute the kernel. In this case a C++ functor
    // transform applies a functor to the input parameters
    // and leaves the result in the last parameter
    thrust::transform(encodingOfSubset.begin(), encodingOfSubset.end(),
                        is_solution.begin(), ef);
    // cout << "Performed transformation." << endl;
    int result = thrust::reduce(
                            is_solution.begin(),is_solution.end(),
                            0,
                            thrust::maximum());

  cout << "The maximum number of nodes in the independent set is: " << result << endl;
  thrust::device_vector<int>::iterator iter;

  iter = thrust::find(thrust::device, is_solution.begin(), is_solution.end(), result);
  int index = iter - is_solution.begin();
  printResults(index,n);
  return 0;
}

Overwriting max_ind_set_thrust.cu


In [49]:
!nvcc -Icccl/thrust -Icccl/libcudacxx/include -Icccl/cub max_ind_set_thrust.cu -o max_ind_set_thrust -arch sm_75


In [28]:
%%writefile k16.txt
16
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

Writing k16.txt


In [50]:
!time ./max_ind_set_thrust < k16.txt

The size of the graph is: 16
The number of rows in the adjacency matrix is: 16
The number of columns in the adjacency matrix is: 16
The adjacency matrix is : 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 
The maximum number of nodes in the independent set is: 1
An independent set with the maximum number of nodes is: 
0 

real	0m0.135s
user	0m0.017s
sys	0m0.110s


In [51]:
%%writefile test16b.txt
16
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

Overwriting test16b.txt


In [52]:
!time ./max_ind_set_thrust < test16b.txt

The size of the graph is: 16
The number of rows in the adjacency matrix is: 16
The number of columns in the adjacency matrix is: 16
The adjacency matrix is : 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 
The maximum number of nodes in the independent set is: 6
An independent set with the maximum number of nodes is: 
0 1 2 3 4 5 

real	0m0.133s
user	0m0.017s
sys	0m0.109s


In [44]:
%%writefile test16c.txt
16
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0

Writing test16c.txt


In [53]:
!time ./max_ind_set_thrust < test16c.txt

The size of the graph is: 16
The number of rows in the adjacency matrix is: 16
The number of columns in the adjacency matrix is: 16
The adjacency matrix is : 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 
The maximum number of nodes in the independent set is: 6
An independent set with the maximum number of nodes is: 
0 1 2 3 4 5 

real	0m0.136s
user	0m0.018s
sys	0m0.110s


In [46]:
%%writefile k20.txt
20
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

Writing k20.txt


In [54]:
!time ./max_ind_set_thrust < k20.txt

The size of the graph is: 20
The number of rows in the adjacency matrix is: 20
The number of columns in the adjacency matrix is: 20
The adjacency matrix is : 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 
The maximum number of